In [167]:
import pandas as pd

filteredDf = pd.read_csv('attribute_list_filtered.csv')
filteredDf.head()

,file_name,hair_color,eyeglasses,smiling,young,human
0,1,1,-1,1,1,-1
1,2,4,-1,1,1,1
2,3,5,-1,1,-1,-1
3,7,2,-1,1,1,-1
4,8,3,-1,1,1,-1


In [3]:
from sklearn.preprocessing import OneHotEncoder

#one-hot-enconding:
def get_one_hot():
    df = pd.read_csv('attribute_list_filtered.csv')
    fileNameDf = df['file_name']
    df = df.drop('file_name', 1)
    enc = OneHotEncoder(categories='auto')
    enc.fit(df)
    onehotlabels = enc.transform(df).toarray()
    print(enc.get_feature_names())
    print(onehotlabels.shape)
    oneHotCols = ["hair_-1", "hair_0", "hair_1", "hair_2", "hair_3", "hair_4",
              "hair_5", "eyeglasses_-1", "eyeglasses_1", "smiling_-1", "smiling_1", 
              "young_-1", "young_1", "human_-1", "human_1"]
    oneHotDf = pd.DataFrame(onehotlabels, columns = oneHotCols)
    oneHotDf = pd.concat([fileNameDf, oneHotDf], axis=1)
    oneHotDf.to_csv('attribute_list_filtered_onehot.csv', index=False)
    return oneHotDf

oneHotDf = get_one_hot()

['x0_-1' 'x0_0' 'x0_1' 'x0_2' 'x0_3' 'x0_4' 'x0_5' 'x1_-1' 'x1_1' 'x2_-1'
 'x2_1' 'x3_-1' 'x3_1' 'x4_-1' 'x4_1']
(4565, 15)


In [4]:
len(oneHotDf)

4565

In [5]:
#data augmentation to prevent over-fitting example:
'''https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'''
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('./images/2.png')
img_arr = img_to_array(img) #shape: (3, 256, 256)
img_arr = img_arr.reshape((1,)+img_arr.shape) #adds extra dimension, shape: (1,3,256,256)

#generate batches of randomly transformed images, save in transformed dir
if os.path.isdir('transformed')==False:
    os.mkdir('transformed')

i = 0
flow = datagen.flow(img_arr, batch_size=1,
                    save_to_dir='transformed', 
                    save_prefix='2', save_format='png')
for batch in flow:
    i+=1
    if i>20:
        break

Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: './images/2.png'

In [7]:
#splitting into train, validation, and test sets:
import numpy as np
def split_dataset_random(df, validation_split=0.2, test_split=0.2, seed=0):
    np.random.seed(seed) #if the same seed is used the same random split will be produced
    train_split = 1 - validation_split - test_split
    indeces_array = [int(train_split*len(df)), int((1-test_split)*len(df))]
    '''example, [2, 3] would, for axis=0, result in

        ary[:2]
        ary[2:3]
        ary[3:]

    '''
    train, validate, test = np.split(df.sample(frac=1), indeces_array)
    return train, validate, test

train, validate, test = split_dataset_random(oneHotDf, seed=123)
print("train length: ", len(train), "| ratio: ", len(train)/len(oneHotDf))
print("validate length: ", len(validate), "| ratio: ", len(validate)/len(oneHotDf))
print("test length: ", len(test), "| ratio: ", len(test)/len(oneHotDf))

train length:  2739 | ratio:  0.6
validate length:  913 | ratio:  0.2
test length:  913 | ratio:  0.2


In [8]:
#Move images to train, test, validate directories according to split:
import shutil
import os

def move_split_images(train, validate, test):
    imagesPath = './images/'
    trainPath = './images/train/'
    validatePath = './images/validate/'
    testPath = './images/test/'
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName)
            trainFilePath = os.path.join(trainPath, fileName)
            validateFilePath = os.path.join(validatePath, fileName)
            testFilePath = os.path.join(testPath, fileName)
            if(df.equals(test)):
                newFilePath = testFilePath
            if(df.equals(train)):
                newFilePath = trainFilePath
            if(df.equals(validate)):
                newFilePath = validateFilePath
            if os.path.isfile(filePath):
                shutil.move(filePath, newFilePath)
            elif os.path.isfile(testFilePath):
                shutil.move(testFilePath, newFilePath)
            elif os.path.isfile(validateFilePath):
                shutil.move(validateFilePath, newFilePath)
            elif os.path.isfile(trainFilePath):
                shutil.move(trainFilePath, newFilePath)
            else:
                print("File missing: ", fileName)

move_split_images(train, validate, test)

In [60]:
validatePath = './images/validate/'
labels_cols = list(train)[1:]
print(labels_cols)

['hair_-1', 'hair_0', 'hair_1', 'hair_2', 'hair_3', 'hair_4', 'hair_5', 'eyeglasses_-1', 'eyeglasses_1', 'smiling_-1', 'smiling_1', 'young_-1', 'young_1', 'human_-1', 'human_1']


In [172]:
train, validate, test = split_dataset_random(filteredDf, seed=123)
print("train length: ", len(train), "| ratio: ", len(train)/len(oneHotDf))
print("validate length: ", len(validate), "| ratio: ", len(validate)/len(oneHotDf))
print("test length: ", len(test), "| ratio: ", len(test)/len(oneHotDf))

move_split_images(train, validate, test)

train_human = train[['file_name', 'human']]
validate_human = validate[['file_name', 'human']]
test_human = test[['file_name', 'human']]
labels_cols = list(train_human)[1:]

train length:  2739 | ratio:  0.6
validate length:  913 | ratio:  0.2
test length:  913 | ratio:  0.2


In [182]:
#https://jovianlin.io/cat-crossentropy-vs-sparse-cat-crossentropy/

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

K.set_image_dim_ordering('tf')

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [171]:
train.head()

,file_name,hair_color,eyeglasses,smiling,young,human
1388,1532,3,-1,1,1,-1
1436,1587,-1,-1,1,-1,1
550,608,1,-1,1,1,1
2151,2359,-1,1,-1,1,1
2242,2458,2,1,1,1,-1


In [76]:
train_human = train[['file_name', 'human_-1', 'human_1']]
validate_human = validate[['file_name', 'human_-1', 'human_1']]
test_human = test[['file_name', 'human_-1', 'human_1']]
labels_cols = list(train_human)[1:]

In [178]:
print(labels_cols)

['human']


In [183]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

#Takes the dataframe and the path to a directory and 
#generates batches of augmented/normalized data.

trainPath = './images/train/'
validatePath = './images/validate/'
#labels_cols = list(train)[1:]
labels_cols = list(train_human)[1:]

train_gen = train_datagen.flow_from_dataframe(
            dataframe = train_human,
            directory = trainPath,
            x_col = "file_name",
            y_col = 'human',
            has_ext = False,
            target_size=(150,150),
            color_mode='rgb',
            class_mode='binary',
            batch_size=batch_size)

validation_gen = test_datagen.flow_from_dataframe(
            dataframe = validate_human,
            directory = validatePath,
            x_col = "file_name",
            y_col = 'human',
            has_ext = False,
            target_size=(150,150),
            color_mode='rgb',
            class_mode='binary',
            batch_size=batch_size)

Found 2739 images belonging to 2 classes.
Found 913 images belonging to 2 classes.


In [ ]:
from keras.callbacks import EarlyStopping

earlyStopping = EarlyStopping(monitor='val_loss')

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[earlyStopping])

#model.save_weights('.h5')
model.save('human_detection.h5')

Epoch 1/50
85/85 [==============================] - 51s 596ms/step - loss: 0.2952 - acc: 0.9062 - val_loss: 9.1649 - val_acc: 0.0100
Epoch 2/50
60/85 [====================>.........] - ETA: 13s - loss: 0.0275 - acc: 0.9922

In [162]:
testPath = './images/test/'

test_gen = test_datagen.flow_from_dataframe(
        dataframe = test_human,
        directory = testPath,
        x_col = "file_name",
        y_col = labels_cols,
        has_ext = False,
        target_size=(150,150),
        color_mode='rgb',
        class_mode='other',
        batch_size=1,
        shuffle=False)

test_gen.reset()

probabilities = model.predict_generator(test_gen,verbose=1,steps=len(test_gen))

Found 913 images.
913/913 [==============================] - 7s 7ms/step


In [166]:
import numpy as np
class_labels_predicted = np.argmax(probabilities, axis=1)
print(class_labels_predicted)


[1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0
 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1
 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1
 1 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0
 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 0
 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1
 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0
 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0
 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0
 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 1 

In [138]:
n_human = test_human['human_1'].value_counts()
n_not_human = test_human['human_-1'].value_counts()
print(n_human, n_not_human)

0.0    503
1.0    410
Name: human_1, dtype: int64 1.0    503
0.0    410
Name: human_-1, dtype: int64


In [164]:
print(probabilities)

[[2.3307790e-23 1.0000000e+00]
 [1.4481164e-27 1.0000000e+00]
 [1.2297226e-27 1.0000000e+00]
 ...
 [1.0000000e+00 0.0000000e+00]
 [4.0981585e-30 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]]


In [163]:
from sklearn.metrics import confusion_matrix
import numpy as np

inputdf = test_human.iloc[:,1:].values
class_labels_input = np.argmax(inputdf, axis=1)
class_labels_predicted = np.argmax(probabilities, axis=1)

#y_true = np.array([0] * 503 + [1] * 410)
#y_pred = probabilities > 0.5

confusion_matrix(class_labels_input, class_labels_predicted)

array([[281, 222],
       [222, 188]])

In [141]:
print(class_labels_predicted)

[1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0
 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1
 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1
 1 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0
 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 0
 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1
 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0
 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0
 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0
 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 1 

In [142]:
print(len(class_labels_input))
print(len(class_labels_predicted))

913
913


In [144]:
test_human.head()

,file_name,human_-1,human_1
9,14,0.0,1.0
3812,4169,0.0,1.0
1950,2144,1.0,0.0
659,730,1.0,0.0
4067,4457,1.0,0.0
